# 장고를 이용하여 인스타 크롤링 하기

## 앱 생성

In [ ]:
# 기존 장고를 만들었던 mysite디렉토리에서 새로운 앱을 생성해준다
python manage.py startapp insta

In [ ]:
# insta/models.py에 insta클래스 생성
class Insta(models.Model):
    content = models.TextField(null=False)
    date = models.CharField(null=False, max_length=10)
    like = models.IntegerField(null=False, default=0)
    place = models.CharField(null=False, max_length=100)
    tags = models.TextField(null=False)
    create_date = models.DateTimeField(auto_now_add=True)

* models.py 파일을 수정해 주었으므로, settings.py파일에 insta앱의 경로를 설정해준다. apps.py파일이 생성됨을 알 수 있다.

In [ ]:
# settings.py 파일에 다음을 추가
INSTALLED_APPS = [
    "django.contrib.admin",
    "django.contrib.auth",
    "django.contrib.contenttypes",
    "django.contrib.sessions",
    "django.contrib.messages",
    "django.contrib.staticfiles",
    "bbsnote.apps.BbsnoteConfig",
    "common.apps.CommonConfig",
    "insta.apps.InstaConfig",
]

* 이후 터미널 명령에서 makemigrations와 migrate을 진행해준다.

* insta폴더 내에 parser.py를 생성해주고, Jupyter환경에서 인스타 크롤링을 하기 위해 만들었던 함수들을 추가해준다
* 크롬 드라이브를 로드하여 사용자의 아이디와 비빌번호를 입력후 설정한 태그 게시물로 이동하여 클릭을 통해 태그의 첫번재 게시물로 이동
* 게시글 수집을 위한 크롤링함수를 실행하여 수집이 되면 다음 게시글로 이동하는 것을 target변수를 통해 반복

In [ ]:
# insta폴더 내에 parser.py를 생성 후 다음의 내용을 추가
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
import re
from collections import Counter
from bs4 import BeautifulSoup



def insta_crawling(results):
    # 크롬 드라이버를 로드
    driver = webdriver.Chrome('D:/chromedriver/chromedriver.exe')

    # 로그인 페이지로 이동
    driver.get('https://www.instagram.com/accounts/login/')
    time.sleep(2)

    # 사용자 이름과 비밀번호를 입력
    username = driver.find_element(By.NAME, 'username')
    password = driver.find_element(By.NAME, 'password')
    username.send_keys('chuncm@naver.com')
    password.send_keys('970dnswjsqud0402!')

    # 로그인 버튼을 클릭
    login_button = driver.find_element(By.XPATH, '//button[@type="submit"]')
    login_button.click()
    time.sleep(5)

    word = "제주도맛집"
    url = insta_searching(word)
    driver.get(url)

    def insta_searching(word):
    url = 'https://www.instagram.com/explore/tags/' + word
    return url


def select_first(driver):
    # F12를 통해 개발자 도구 페이지로 가서 class명이 '_aagu'인 부분을 살펴보면 게시글에 해당된다
    first = driver.find_element(By.CLASS_NAME, '_aagu')
    first.click()
    time.sleep(3)
    select_first(driver)


    # 게시글 수집을 위해 크롤링 할 게시글 수 지정
    target = 10
    for i in range(target):
        try:
            data = get_content(driver)
            results.append(data)
            move_next(driver)
        except:
            # 게시글 수집에 오류 발생 시(네트워크 문제 등의 이유로), 2초 대기 후, 다음게시물로 넘어가도록 지정
            time.sleep(2)
            move_next(driver)
            
# 맛집 크롤링
def get_content(driver):
    # 현재 페이지 html정보 가져오기
    html = driver.page_source
    soup = BeautifulSoup(html, 'lxml')
    # 본문내용 가져오기
    try:
        content = soup.select('div._a9zs > h1')[0].text
    except:
        content = ' '
    # #으로 시작하여, #뒤에 연속된 문자(공백이나 #, \기호가 아닌 경우)를 모두 찾아서 리스트 형태로 저장
    tags = re.findall(r'#[^\s#,\<]+', content)
    date = soup.select('time._aaqe')[0]['datetime'][:10]
    # 좋아요 수 가져오기
    try:
        # 태그명이 div, class명이 다음과 같은 태그 바로 아래에 있는 span태그 모두 선택
        like = soup.select('div._ae2s._ae3v._ae3w > section._ae5m._ae5n._ae5o > div > div > span > a > span > span')[0].text
    except:
        like = 0
    # 위치(장소)정보 가져오기
    try:
        # div태그 중 class명이 _aaqm인 것을 선택해 화면에 노출되는 첫 번째 태그의 text를 가져옴
        place = soup.select('div._aaqm')[0].text
    except:
        place = ''
    # 수집한 정보 저장하기
    data = [content, date, like, place, tags]
    return data
get_content(driver)


# 다음 게시글 열기
def move_next(driver):
    right = driver.find_element(By.CLASS_NAME, '_aaqg')
    right.click()
    time.sleep(3)
move_next(driver)

* 경로 지정을 위해 config파일과 insta폴더에 insta의 경로를 지정해준다.

In [ ]:
# config/urls.py에 insta의 경로 추가
urlpatterns = [
    path("admin/", admin.site.urls),
    path("bbsnote/", include('bbsnote.urls')),
    path("common/", include('common.urls')),
    path('', views.index, name='index'),
    # 인스타라는 경로로 들어오면 'insta.urls'의 URL패턴 목록이 포함된다
    path('insta/', include('insta.urls')),
]

In [ ]:
# insta폴더 안에 urls.py파일 생성 후 다음의 내용 추가
from django.urls import path
from . import views

app_name = 'insta'

urlpatterns = [
    # 첫번째 URL패턴이 빈 문자열과 매칭 후, views.index 뷰 함수와 매칭
    path('', views.index, name='index'),
]

* 경로에 지정한 뷰함수를 정의하기 위해 views.py 파일에 index함수를 추가해준다.

In [ ]:
# insta/views.py파일에 다음의 함수 추가
from django.shortcuts import render
from .parser import *
from .models import Insta

def insta(request):
    results = []
    insta_crawling(results)
    
    for row in results:
        insta = Insta(content=row[0], date=row[1], like=row[2], place=row[3], tags[4])
        insta.save()

* 터미널 명령 runserver를 통해 서버에 insta항목이 생긴것을 확인한다.